# **A simple web application with nglview and periodic table widget**

* textarea to input name of the molecules (ipywidgets)
* obtain molecular structure from ase.build.molecule (ase)
* show the structure with nglview (nglview)
* highlight the elements of the strucutre in the periodic table (widget_periodictable)
* use Voila and OSSCAR template to present the web app (voila and voila-osscar-template)

In [8]:
from ase.build import molecule
from widget_periodictable import PTableWidget
import nglview as nv
from ipywidgets import HBox, VBox, Textarea, Button, Layout
import numpy as np
from vapory import *
from numpy.linalg import norm
from copy import deepcopy

In [9]:
colors = {'H': [255,255,255],
         'He': [217,255,255],
         'Li': [204,128,255],
         'Be': [194,255,0],
         'B': [255,181,181],
         'C': [144,144,144],
         'N': [48,80,248],
         'O': [255,13,13],
         'F': [144,224,80],
         'Ne': [179,227,245],
         'Na': [171,92,242],
         'Mg': [138,255,0],
         'Al': [191,166,166],
         'Si': [240,200,160],
         'P': [255,128,0],
         'S': [255,255,48],
         'Cl': [31,240,31],
         'Ar': [128,209,227],
         'K': [143,64,212],
         'Ca': [61,255,0]}

radius = {'H': 0.53,
         'He': 0.31,
         'Li': 1.67,
         'Be': 1.12,
         'B': 0.87,
         'C': 0.67,
         'N': 0.56,
         'O': 0.48,
         'F': 0.42,
         'Ne': 0.38,
         'Na': 1.90,
         'Mg': 1.45,
         'Al': 1.18,
         'Si': 1.11,
         'P': 0.98,
         'S': 0.88,
         'Cl': 0.79,
         'Ar': 0.71,
         'K': 2.43,
         'Ca': 1.94}

In [10]:
aa = molecule("C60")
aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
aa.center()
aa.pbc=True
view = nv.show_ase(aa)
view.add_unitcell()
view.control.zoom(0.2)
view.add_ball_and_stick(aspectRatio=4)
view.camera='perspective'

In [11]:
w = Textarea(
    value='C60',
    placeholder='Type your molecule',
    description='Molecule:',
    disabled=False,
    layout=Layout(width='28%', height='27px')
)

In [12]:
PTable = PTableWidget(states=1, selected_colors = ['red'], selected_elements = {'C':0})

In [13]:
def on_button_click(b):
    global view, aa
    aa = molecule(w.value)
    aa.set_cell([[15, 0, 0], [0, 15, 0], [0, 0, 15]])
    aa.center()
    aa.pbc=True
    for comp_id in view._ngl_component_ids:
        view.remove_component(comp_id)
    view.add_component(nv.ASEStructure(aa))
    view.clear()
    view.add_ball_and_stick(aspectRatio=4)
    view.add_unitcell()
    view.control.zoom(0.2)
    PTable.selected_elements = {key: 0 for key in list(dict.fromkeys(aa.get_chemical_symbols()))}

def povray_render(b):
    A = view._camera_orientation
    A = np.array(A)
    A=A.reshape(4,4)
    A=np.transpose(A)

    zfactor = norm(A[0, 0:3])
    A[0:3, 0:3] = A[0:3, 0:3]/zfactor
    
    bb = deepcopy(aa);

    for i in bb:
        a = np.array([i.x, i.y, i.z])
        a = a + A[0:3, 3];
        w = A[0:3, 0:3].dot(a)
        i.x = -w[0]
        i.y = w[1]
        i.z = w[2]    

    camera = Camera( 'location', [0, 0, -zfactor/1.5], 'look_at', [0.0, 0.0, 0.0])

    light1 = LightSource([0, 0, -60.0], 'color',  [1.5, 1.5, 1.5])
    light2 = LightSource([-60.0, 0, 0], 'color',  [1, 1, 1])
    light3 = LightSource([0, -60.0, 0], 'color',  [1, 1, 1])

    #light = LightSource(-A[2][0:3], [1.3, 1.3, 1.3])
    wall = Plane([0, 0, 200], 20, Texture(Pigment('color', [1, 1, 1])))

    spheres = [];

    for i in bb:
        sphere = Sphere( [i.x, i.y, i.z], radius[i.symbol], 
                        Texture(Pigment( 'color', np.array(colors[i.symbol])/255)))
        spheres.append(sphere)


    bonds = [];
    for x, i in enumerate(bb):
        for j in bb[x+1:]:
            v1 = np.array([i.x, i.y, i.z])
            v2 = np.array([j.x, j.y, j.z])

            if norm(v1-v2) < 1.5:
                bond = Cylinder(v1, v2, 0.2, Pigment('color', [0, 0, 1]),Finish('phong', 0.8,'reflection', 0.5))
                bonds.append(bond)

    scene = Scene( camera, objects= [light1, wall] + spheres + bonds)
    scene.render("nglview.png", width=1000, height=1000, antialiasing=0.001)

    
b = Button(description = 'Update')
br = Button(description = 'Render')
b.on_click(on_button_click)
br.on_click(povray_render)

In [14]:
display(HBox([w, b, br]), VBox([view, PTable]))